# VIEWERTOOL 1.0

06.05.2020
namd psf, dcd
python 3.6.7 
requirements (nanodesign_dietz, MDAnalysis, numpy, mrcfile, nglview, ipywidgets, pyqt5)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%gui qt

In [25]:
import linkerapp as linkerAPP
import linkerappUtil as Util

# select  project
select working directoy and required files. each cell opens a seperate FileDialog Popup

In [15]:
# select a working directory
wd = Util.gui_folderDialog()

In [16]:
# select a cadnano deisgn file
json = Util.gui_fileDialog(wd=wd, filter="caDNAno (*.json)")

In [18]:
psf = Util.gui_fileDialog(wd=wd, filter="topology (*.psf)")

In [19]:
pdb = Util.gui_fileDialog(wd=wd, filter="coordinates (*.dcd *.pdb)")

In [20]:
mrc = Util.gui_fileDialog(wd=wd, filter="cryo-map (*.mrc)")

In [22]:
seq = Util.gui_fileDialog(wd=wd, filter="sequence file (*.seq *.txt)")

In [33]:
json="/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.json"
pdf="/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.psf"
pdb="/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.pdb"
mrc="/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.mrc"
seq="/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.seq"

/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.json
/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.psf
/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.pdb
/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.mrc
/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.seq


In [23]:
files = Util.Files(json, psf, pdb, mrc, seq)

In [34]:
viewer = linkerAPP.Viewer(wd, files=files)

In [37]:
viewer.linker

Linker(project=Project(input=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine'), output=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/analysis'), name='TTnative', files=Files(json=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.json'), psf=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.psf'), coor=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__TTnative/a-iii-e-TTnative-074__refine/TTnative.pdb'), mrc=PosixPath('/Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/

# select helices and bases
either use the widget or specify custom selection:
* selection -> tuple containing to lists ([helix-ids],[base_ids])
* context number of voxels arround a selection for map zoning

## option 1: selection with widget 
(correct depiction for squarelattice)

In [28]:
sliders = viewer.make_sliders()

Box(children=(VBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, layout=Layout(height='30p…

In [38]:
helixandbase, context = viewer.eval_sliders(*sliders)
atoms_selection, atoms_scaffold, atoms_staple, color_dict = viewer.select_by_helixandbase(*helixandbase)


## option 2: custom selection

In [39]:
# or generate stomselection using mdAnalysis
#TODO: color not retreivable
#HINT: atomselections are combinable

([55, 54, 53, 48, 49, 50], range(33, 115))

#  save selection
-> name-temp.pdb/.mrc
-> name-temp_chim.pdb

write a pdb and an mrc of the selected atoms/regions to disk. (note: mrc file is overwritten whenever a new selection is made). name-temp_chim.pdb is corrected for use in chimera

* last: if True only write the final frame

In [ ]:
#create subset MRC-file (TODO: additional info)
Util.mrc(atoms_selection, path=wd, context=context, cut_box=True)

In [ ]:
#create subset pdb-file (chimeraX compatible) #TODO: vmd compatibility
viewer.writepdb(atoms_selection, single_frame=True, frame=-1)

#  show selection
* isovalue: isosurface threshold
* backbone: true -> only backbone atoms displayed (faster); false -> all
* color_dict: None -> no staple coloring (significantely faster)

use the play button to show a movie of of the fitting or move the cursor to a specific timestep 

In [31]:
#color_dict = None
Util.fit_widget(atoms_selection, atoms_scaffold, atoms_staple, path=path, isolevel=1.35, backbone=False, color_dict=None)


ValueError: you must provide file extension if using file-like object or text content